.

Data_Engineering_TIL(20200725)

Study Program : Docker 기반의 DevOps 인프라 구축 WORKSHOP

관련 URL : https://www.fastcampus.co.kr/dev_workshop_devops

학습자료 출처 : http://www.authorstream.com/Presentation/k16wire-4350641-devops-docker-day3-v7/

#### 1. 기초개념

1) Networking model

- 도커가 시작될때 호스트 머신에 docker0 이라고 부르는 가상 인터페이스를 생성한다.


- docker0에 사설 IP가 랜덤하게 배정된다.


![image](https://user-images.githubusercontent.com/41605276/88762444-2abf9480-d1ac-11ea-8e1f-29853ba76bf7.png)


2) 네트워크 유형

- bridge : 단일 호스트내에서 컨테이너를 연결해주는 네트워크

ex) docker0

- overlay : 멀티 호스트간에 연결해주는 네트워크


3) bridge network

- 컨테이너는 동일 호스트내에 위치해야 한다.


- 사용자 정의 bridge 네트워크에 포함된 컨테이너는 컨테이너 이름으로 통신 가능


4) overlay network

- 오버레이 네트워크 요건

키-벨류 스토어 : Consul, Etcd, Zookeeper


키-벨류 스토어에 연결된 호스트 클러스터


커널버전 3.16 이상


각 호스트 도커 엔진 설정


- consul.io

HashiCorp에서 만든 오픈소스로 서비스 디스커버리나 설정 관련 기능을 구현할때 사용할 수 있는 서비스 메시 솔루션


주요특징


서비스 디스커버리


헬스체크


키벨류 스토어


시큐어 서비스 스토어


- 도커 호스트 간의 네트워크


- 다른 도커 호스트에서 각각 실행중이 컨테이너들이 서로 통신할 수 있게 해준다.


- 도커 엔진은 overlay 네트워크 드라이버를 통해 멀티 호스트 네트워크 지원


![image](https://user-images.githubusercontent.com/41605276/88764299-aec74b80-d1af-11ea-87bc-11e5c5d19a2d.png)


- 포트 오픈

TCP 2377 : 클러스터 관리


TCP/UDP 7946 : 노드간 커뮤니케이션


UDP 4789 : 오버레이 네트워크 트래픽


#### 2. bridge network 실습

![image](https://user-images.githubusercontent.com/41605276/88762742-ba654300-d1ac-11ea-981c-f999c4e29540.png)

In [ ]:
[ec2-user@ip-10-1-10-112 ~]$ sudo docker network ls
NETWORK ID          NAME                DRIVER              SCOPE
f6d7bf5ebaa7        bridge              bridge              local
c42531cf5948        host                host                local
5908948ffe8d        none                null                local

[ec2-user@ip-10-1-10-112 ~]$ sudo docker network inspect bridge
[
    {
        "Name": "bridge",
        "Id": "f6d7bf5ebaa736724cb863a2d25a9067b6ebb0d934d14d4dce1604907c759a6f",
        "Created": "2020-07-25T06:00:45.422629052Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/16"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {
            "com.docker.network.bridge.default_bridge": "true",
            "com.docker.network.bridge.enable_icc": "true",
            "com.docker.network.bridge.enable_ip_masquerade": "true",
            "com.docker.network.bridge.host_binding_ipv4": "0.0.0.0",
            "com.docker.network.bridge.name": "docker0",
            "com.docker.network.driver.mtu": "1500"
        },
        "Labels": {}
    }
]

red 네트워크 생성
[ec2-user@ip-10-1-10-112 ~]$ sudo docker network create red
7cf96ba221fecfd33dd29759efb4bd2da88a3889841a936aeb651e82439bcbf4

blue 네트워크 생성
[ec2-user@ip-10-1-10-112 ~]$ sudo docker network create blue
1f5435e3f8568f506d6d18982da88826537a76bcd88f6093886f05e81bf0e89f

[ec2-user@ip-10-1-10-112 ~]$ sudo docker network ls
NETWORK ID          NAME                DRIVER              SCOPE
1f5435e3f856        blue                bridge              local
f6d7bf5ebaa7        bridge              bridge              local
c42531cf5948        host                host                local
5908948ffe8d        none                null                local
7cf96ba221fe        red                 bridge              local

red 네트워크에 ubuntu1을 추가
[ec2-user@ip-10-1-10-112 ~]$ sudo docker run -itd --net=red --name ubuntu1 ubuntu
872afa0613991a2db883a72a87aa4fb4f116e742844b378eefdfd62704386618

blue 네트워크에 ubuntu3을 추가
[ec2-user@ip-10-1-10-112 ~]$ sudo docker run -itd --net=blue --name ubuntu3 ubuntu
5f6a9da48f2bc435878d89f49660ef718381d86cc5a364a7c7e382166527363a

red 네트워크 확인
[ec2-user@ip-10-1-10-112 ~]$ sudo docker network inspect red
[
    {
        "Name": "red",
        "Id": "7cf96ba221fecfd33dd29759efb4bd2da88a3889841a936aeb651e82439bcbf4",
        "Created": "2020-07-25T06:37:01.88911991Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "872afa0613991a2db883a72a87aa4fb4f116e742844b378eefdfd62704386618": {
                "Name": "ubuntu1",
                "EndpointID": "d40c0790c687a0f97283e9325dfe4056a1944ca8f317faed5b18e0e89f9a98a5",
                "MacAddress": "02:42:ac:12:00:02",
                "IPv4Address": "172.18.0.2/16",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]

blue 네트워크 확인
[ec2-user@ip-10-1-10-112 ~]$ sudo docker network inspect blue
[
    {
        "Name": "blue",
        "Id": "1f5435e3f8568f506d6d18982da88826537a76bcd88f6093886f05e81bf0e89f",
        "Created": "2020-07-25T06:37:10.607242774Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.19.0.0/16",
                    "Gateway": "172.19.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "5f6a9da48f2bc435878d89f49660ef718381d86cc5a364a7c7e382166527363a": {
                "Name": "ubuntu3",
                "EndpointID": "caac65b64138fada05ba7ddd39218aad7dc6615adf4cc6999386768f808c1a85",
                "MacAddress": "02:42:ac:13:00:02",
                "IPv4Address": "172.19.0.2/16",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]

red 네트워크에 ubuntu2 추가
[ec2-user@ip-10-1-10-112 ~]$ sudo docker run -itd --net=red --name ubuntu2 ubuntu
9d8ae7baf4e54c47923a50369b613ea6807d32ee251badd927b783c49ad718da

blue 네트워크에 ubuntu4 추가 및 데몬모드를 해제하여 실행
[ec2-user@ip-10-1-10-112 ~]$ sudo docker run -it --net=blue --name ubuntu4 ubuntu
root@2234a98762fa:/# cat /etc/hosts
127.0.0.1       localhost
::1     localhost ip6-localhost ip6-loopback
fe00::0 ip6-localnet
ff00::0 ip6-mcastprefix
ff02::1 ip6-allnodes
ff02::2 ip6-allrouters
172.19.0.3      2234a98762fa
root@2234a98762fa:/# apt-get update -y
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [107 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [51.3 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [38.7 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [1078 B]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [187 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [111 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [98.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:12 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [401 kB]
Get:14 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [179 kB]
Get:15 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [17.4 kB]
Get:16 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [38.7 kB]
Get:17 http://archive.ubuntu.com/ubuntu focal-backports/universe amd64 Packages [3209 B]
Fetched 14.3 MB in 5s (2620 kB/s)
Reading package lists... Done
root@2234a98762fa:/# apt-get install inetutils-ping -y
Reading package lists... Done
Building dependency tree
Reading state information... Done
The following additional packages will be installed:
  libidn11 netbase
The following NEW packages will be installed:
  inetutils-ping libidn11 netbase
0 upgraded, 3 newly installed, 0 to remove and 4 not upgraded.
Need to get 120 kB of archives.
After this operation, 657 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 netbase all 6.1 [13.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libidn11 amd64 1.33-2.2ubuntu2 [46.2 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 inetutils-ping amd64 2:1.9.4-11 [60.8 kB]
Fetched 120 kB in 1s (97.9 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package netbase.
(Reading database ... 4122 files and directories currently installed.)
Preparing to unpack .../archives/netbase_6.1_all.deb ...
Unpacking netbase (6.1) ...
Selecting previously unselected package libidn11:amd64.
Preparing to unpack .../libidn11_1.33-2.2ubuntu2_amd64.deb ...
Unpacking libidn11:amd64 (1.33-2.2ubuntu2) ...
Selecting previously unselected package inetutils-ping.
Preparing to unpack .../inetutils-ping_2%3a1.9.4-11_amd64.deb ...
Unpacking inetutils-ping (2:1.9.4-11) ...
Setting up libidn11:amd64 (1.33-2.2ubuntu2) ...
Setting up netbase (6.1) ...
Setting up inetutils-ping (2:1.9.4-11) ...
Processing triggers for libc-bin (2.31-0ubuntu9) ...
root@2234a98762fa:/# ping
ping: missing host operand
Try 'ping --help' or 'ping --usage' for more information.

핑으로 ubuntu 2 때리면 반응이 없는데 왜냐하면 red 네트워크에 있기 때문이다.

root@2234a98762fa:/# ping ubuntu2
ping: unknown host

반면에 ubuntu 3를 핑을 때리면 같은 blue 네트워크에 있기 때문에 반응이 온다.
    
root@2234a98762fa:/# ping ubuntu3
PING ubuntu3 (172.19.0.2): 56 data bytes
64 bytes from 172.19.0.2: icmp_seq=0 ttl=255 time=0.091 ms
64 bytes from 172.19.0.2: icmp_seq=1 ttl=255 time=0.078 ms
64 bytes from 172.19.0.2: icmp_seq=2 ttl=255 time=0.076 ms
64 bytes from 172.19.0.2: icmp_seq=3 ttl=255 time=0.080 ms
64 bytes from 172.19.0.2: icmp_seq=4 ttl=255 time=0.074 ms
64 bytes from 172.19.0.2: icmp_seq=5 ttl=255 time=0.075 ms
64 bytes from 172.19.0.2: icmp_seq=6 ttl=255 time=0.078 ms
64 bytes from 172.19.0.2: icmp_seq=7 ttl=255 time=0.076 ms
64 bytes from 172.19.0.2: icmp_seq=8 ttl=255 time=0.076 ms
64 bytes from 172.19.0.2: icmp_seq=9 ttl=255 time=0.141 ms
64 bytes from 172.19.0.2: icmp_seq=10 ttl=255 time=0.081 ms
64 bytes from 172.19.0.2: icmp_seq=11 ttl=255 time=0.089 ms
64 bytes from 172.19.0.2: icmp_seq=12 ttl=255 time=0.077 ms
64 bytes from 172.19.0.2: icmp_seq=13 ttl=255 time=0.077 ms
64 bytes from 172.19.0.2: icmp_seq=14 ttl=255 time=0.076 ms
64 bytes from 172.19.0.2: icmp_seq=15 ttl=255 time=0.078 ms
64 bytes from 172.19.0.2: icmp_seq=16 ttl=255 time=0.080 ms
^C--- ubuntu3 ping statistics ---
17 packets transmitted, 17 packets received, 0% packet loss
round-trip min/avg/max/stddev = 0.074/0.083/0.141/0.000 ms


동일한 ec2의 다른 터미널 띄워서 아래와 같이 명령어 실행
blue와 red 네트워크를 연결
[ec2-user@ip-10-1-10-112 ~]$ sudo docker network connect blue ubuntu1


다시 이전 터미널로 돌아와서 ubuntu1을 때리면 핑 반응이 오는 걸 확인할 수 있다.
root@2234a98762fa:/# ping ubuntu1
PING ubuntu1 (172.19.0.4): 56 data bytes
64 bytes from 172.19.0.4: icmp_seq=0 ttl=255 time=0.115 ms
64 bytes from 172.19.0.4: icmp_seq=1 ttl=255 time=0.074 ms
64 bytes from 172.19.0.4: icmp_seq=2 ttl=255 time=0.072 ms
64 bytes from 172.19.0.4: icmp_seq=3 ttl=255 time=0.071 ms
64 bytes from 172.19.0.4: icmp_seq=4 ttl=255 time=0.073 ms
64 bytes from 172.19.0.4: icmp_seq=5 ttl=255 time=0.073 ms
64 bytes from 172.19.0.4: icmp_seq=6 ttl=255 time=0.077 ms
64 bytes from 172.19.0.4: icmp_seq=7 ttl=255 time=0.074 ms
64 bytes from 172.19.0.4: icmp_seq=8 ttl=255 time=0.074 ms
64 bytes from 172.19.0.4: icmp_seq=9 ttl=255 time=0.078 ms
64 bytes from 172.19.0.4: icmp_seq=10 ttl=255 time=0.072 ms
64 bytes from 172.19.0.4: icmp_seq=11 ttl=255 time=0.078 ms
^C--- ubuntu1 ping statistics ---
12 packets transmitted, 12 packets received, 0% packet loss
round-trip min/avg/max/stddev = 0.071/0.078/0.115/0.000 ms
root@2234a98762fa:/#